In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-modelling/Churn_Modelling.csv


In [2]:
df = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
x = df.iloc[:,3:13]
y = df.iloc[:,13]

In [5]:
geo = pd.get_dummies(x['Geography'],drop_first=1)
gender = pd.get_dummies(x['Gender'],drop_first=1)

In [7]:
x = pd.concat([x,geo,gender],axis=1)

In [8]:
x = x.drop(['Geography','Gender'],axis=1)

In [9]:
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

In [15]:
clf = Sequential()

In [16]:
clf.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=11))

In [17]:
clf.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))

In [18]:
clf.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [19]:
clf.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
model = clf.fit(x_train,y_train,batch_size=10,validation_split=0.33,epochs=100)

Epoch 1/100
536/536 [==============================] - 3s 3ms/step - loss: 0.6320 - accuracy: 0.6442 - val_loss: 0.5115 - val_accuracy: 0.7959
Epoch 2/100
536/536 [==============================] - 2s 3ms/step - loss: 0.4759 - accuracy: 0.7957 - val_loss: 0.4507 - val_accuracy: 0.7955
Epoch 3/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4387 - accuracy: 0.7987 - val_loss: 0.4338 - val_accuracy: 0.7982
Epoch 4/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4243 - accuracy: 0.8050 - val_loss: 0.4226 - val_accuracy: 0.8069
Epoch 5/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4122 - accuracy: 0.8168 - val_loss: 0.4105 - val_accuracy: 0.8209
Epoch 6/100
536/536 [==============================] - 1s 2ms/step - loss: 0.3978 - accuracy: 0.8285 - val_loss: 0.3970 - val_accuracy: 0.8266
Epoch 7/100
536/536 [==============================] - 2s 3ms/step - loss: 0.3826 - accuracy: 0.8369 - val_loss: 0.3834 - val_accuracy: 0.8353

In [29]:
y_pred = clf.predict(x_test)
y_pred = (y_pred>0.5)

63/63 [==============================] - 0s 1ms/step


In [30]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,y_test)

In [31]:
score

0.8575

In [32]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [33]:
cm

array([[1505,   90],
       [ 195,  210]])